In [ ]:
%load_ext autoreload
%autoreload 2
import torchtext

import sys
#from gensim.utils import tokenize
import pandas as pd
import numpy as np
import re
import transformers
#from sklearn.preprocessing import LabelEncoder
import os
import re
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)
import tensorflow_hub as hub
from utils import *
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import plotly.express as px
import plotly.graph_objects as go
#import bert
import torch
import torch.nn as nn
#import gensim
import sys
import string
import logging
import torch.optim as optim
import random
#import matplotlib.pyplot as plt
#from gensim.models import word2vec

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TensorFlow version:  2.18.0


In [ ]:
%

In [8]:
colnames=['image_path', 'text', 'food']
train = pd.read_csv('train_titles.csv', names=colnames, header=None, sep = ',', index_col=['image_path'])
test = pd.read_csv('test_titles.csv', names=colnames, header=None, sep = ',', index_col=['image_path'])
# Sort values by 'image_path'
test = test.sort_values('image_path')
train = train.sort_values('image_path')
train.head()

,text,food
image_path,,
apple_pie_0.jpg,Apple pie - Wikipedia,apple_pie
apple_pie_100.jpg,Glazed Apple Pie Squares Recipe | Taste of Home,apple_pie
apple_pie_101.jpg,Mock Apple Pie Recipe - Allrecipes.com,apple_pie
apple_pie_102.jpg,Crock-Pot Ladies Crock-Pot Apple Pie Moonshine,apple_pie
apple_pie_104.jpg,All-Star Apple Pie Recipe | Taste of Home,apple_pie


In [9]:
# Check the shapes
print("train samples:",train.shape[0])
print("test samples:",test.shape[0])
vec_preprocess_text = np.vectorize(preprocess_text)
processed_train = vec_preprocess_text(train.text.values.tolist() + test.text.values.tolist())
print(train.text.values.tolist()[0])
print(processed_train[0])

train samples: 67972
test samples: 22716
Apple pie - Wikipedia
apple pie wikipedia


In [ ]:
%pip list


In [12]:
%pip install torchtext -U
%pip list

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\vm-user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Package                  Version
------------------------ -----------
absl-py                  2.1.0
aiobotocore              2.13.0
aiohttp                  3.9.5
aioitertools             0.11.0
aiosignal                1.3.1
annotated-types          0.7.0
asttokens                2.4.1
astunparse               1.6.3
attrs                    23.2.0
azure-common             1.1.28
azure-core               1.30.2
azure-identity           1.17.0
azure-mgmt-compute       31.0.0
azure-mgmt-core          1.4.0
azure-mgmt-network       25.4.0
blis                     1.0.1
bokeh                    3.4.1
botocore                 1.34.106
cachetools               5.3.3
catalogue                2.0.10
certifi                  2024.2.2
cffi                     1.16.0
charset-normalizer       3.3.2
click                    8.1.7
cloudpathlib             0.20.0
cloudpickle              3.0.0
colorama                 0.4.6
comm                     0.2.2
confection               0.1.5
contourpy     

### Sequence Length Histograms

In [ ]:
#listData = [list(tokenize(processed_train[i])) for i in range(processed_train.shape[0])]
#lengths = [len(listData[i]) for i in range(len(listData))]
#fig,ax = plt.subplots(figsize = (7,5))
#ax.hist(lengths,bins = range(0,30,1))
#ax.set_ylabel('Counts', fontsize = 11)
#ax.set_xlabel('Number of words', fontsize =11)
#ax.set_title('Sentence lengths in input data', fontsize = 11)

### Fit Word2vec on entire dataset

In [13]:
"""
dim = 100                 
w2vModel = word2vec.Word2Vec(listData, vector_size=dim,min_count=1)
# add default key, that is, value for unknown key, may happen during testing
w2vModel.wv.add_vector(0,np.zeros((dim)))
w2v_weights = w2vModel.wv.vectors

print(w2vModel.wv.most_similar('barbecue',topn=3))
print(w2vModel.wv.most_similar('apple',topn=3))
print(w2vModel.wv.most_similar('falafel',topn=3))
"""

import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
import io
import spacy
from collections import Counter


#en_tokenizer = get_tokenizer('spacy', language='en')
#nlp = spacy.load("en_core_web_sm")
try:
    spacy_en = spacy.load("en_core_web_sm")
except IOError:
    os.system("python -m spacy download en_core_web_sm")
    spacy_en = spacy.load("en_core_web_sm")

def build_vocab(data, tokenizer):
    counter = Counter()
    for string_ in data:
        counter.update(tokenizer.tokenizer(str(string_)))
    
    return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

vocab = build_vocab(processed_train, spacy_en)

OSError: [WinError 127] The specified procedure could not be found

In [ ]:
#print(vocab.__len__() )
print(vocab.itos[5])
dict = vocab.stoi
dict[vocab.itos[5]]
#print(vocab.itos[5].type)
vocab.set_default_index(vocab['<unk>'])


### Map Captions to Tokens to IDs and Assign Labels

In [ ]:
def token2idNew(listData, maxLen, vocab):
    ids = torch.zeros( (len(listData),maxLen) )
    # set equal to id of 0/pad vector
    ids[:,:] = torch.tensor(w2vModel.wv.key_to_index[0])
    for i in range(len(listData)):
        for j in range(min([maxLen,len(listData[i])])):
            ids[i,j] = torch.tensor(w2vModel.wv.key_to_index[listData[i][j]])
    return ids

In [ ]:
numClasses = 101
# optionally downsample by reducing the number of classes 
#train,test = downSampleData(numClasses, train, test)
countsTrn,countsTst  = classCounts(numClasses, train, test)
processed_train = vec_preprocess_text(train.text.values)
processed_test = vec_preprocess_text(test.text.values)
#print(processed_train[0])
#print(processed_test[0])
# get train and test labels
encoder = LabelEncoder()
encoded_labels_train = encoder.fit_transform(train.food.values)
encoded_labels_test = encoder.fit_transform(test.food.values)
# map train and test tokens to ids
maxLen = 32
#processed_train = [list(tokenize(processed_train[i])) for i in range(processed_train.shape[0])]
#idsTrn = token2id(processed_train, maxLen, w2vModel)
idsTrn = token2idNew(processed_train,maxLen, vocab)
#processed_test = [list(tokenize(processed_test[i])) for i in range(processed_test.shape[0])]
#idsTst = token2id(processed_test, maxLen, w2vModel)
idsTst = token2idNew(processed_test,maxLen, vocab)

In [ ]:
class w2nModel(torch.nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_size, nClasses):
        super(w2nModel, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_size,batch_first=True)
        #self.bn = nn.BatchNorm1d(hidden_size)
        self.drop1 = nn.Dropout(p = 0.5)
        self.FC1 = nn.Linear(hidden_size,256)
        self.drop2 = nn.Dropout(p = 0.5)
        self.FC2 = nn.Linear(256,nClasses)
        self.Relu = torch.nn.ReLU()


    def forward(self, x):
        x = self.word_embeddings(x) # output dimensions is batch size = N x sequence length x feature size
        (x,_) = self.LSTM(x)        
        x = x[:, -1, :] # gives two dimensional output, not three dimensional output
        #x = self.bn(x)
        x = self.drop1(x)               
        x = self.Relu(self.FC1(x))
        x = self.drop2(x)
        x = self.FC2(x)
        return x

### Train Neural Net

In [ ]:
#from torchsummary import summary
torch.device('cuda:0')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#device = 'gpu'
hidden_size = 64
model = w2nModel(vocab_size = w2v_weights.shape[0], 
                 embedding_dim = w2v_weights.shape[1],
                 hidden_size=hidden_size, nClasses = numClasses
                )
#model.word_embeddings.weight.data.copy_(torch.from_numpy(w2v_weights))
#model.word_embeddings.weight.requires_grad=False

labels_train = torch.tensor(encoded_labels_train).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTrn.int(), labels_train)), batch_size=batchSize,
                                         shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
lossVals = []
model = model.to(device)
print(model)

In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader):
        inputs,labels = data
        inputs = inputs.to(device) 
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        lossVals.append(loss.detach().cpu().numpy())
        optimizer.step()
    if epoch%5 == 0:
        print(loss)
print('Finished Training')

In [ ]:
labels_test = torch.tensor(encoded_labels_test).long()
testLoader = torch.utils.data.DataLoader(list(zip(idsTst.int(), labels_test)), batch_size=batchSize,
                                         shuffle=True)
# again no gradients needed
model.eval()
correct_pred = 0
with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs = inputs.to(device) 
        labels = labels.to(device)
        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred += 1

accuracy = 100 * float(correct_pred)/ idsTst.shape[0]
print(accuracy)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=1)
ax.plot(range(len(lossVals)),lossVals)
ax.set_xlabel('Iterations',fontsize = 15)
ax.set_ylabel('Cross Entropy Loss', fontsize = 15)
ax.set_title('Classification Accuracy = {:.2f}%'.format(accuracy),fontsize = 15)
path = 'dim_{}_accry_{:.2f}len_{}_hidden_{}'.format(dim,accuracy, maxLen,hidden_size)
plt.savefig(path+'.pdf')

### Get 5 Most Error-Full Classes

In [ ]:
path = os.getcwd()
checkpoint_path = path+'\_accrcy_{:.2f}_dim_{}.pt'.format(accuracy,dim)
print(checkpoint_path)
torch.save(model, checkpoint_path)

# SVM with LSTM

### Declare LSTM model

In [ ]:
class w2nModelSVM(torch.nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_size, nClasses):
        super(w2nModelSVM, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_size,batch_first=True)

    def forward(self, x):
        x = self.word_embeddings(x)     # output dimensions is batch size = N x sequence length x feature size
        (x,_) = self.LSTM(x)        
        x = x[:, -1, :]                 # gives two dimensional output, not three dimensional output
        return x

torch.device('cuda:0')
modelSVM = w2nModelSVM(vocab_size = w2v_weights.shape[0], 
                 embedding_dim = w2v_weights.shape[1],
                 hidden_size=hidden_size, nClasses = numClasses
                )
modelSVM.word_embeddings.weight.data.copy_(torch.from_numpy(w2v_weights))
modelSVM.word_embeddings.weight.requires_grad=False
modelSVM.LSTM.load_state_dict(model.LSTM.state_dict())
modelSVM.LSTM.requires_grad_=False

### Get LSTM Embeddings for Train and Test Dataset

In [ ]:
labels_train = torch.tensor(encoded_labels_train).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTrn.int(), labels_train)), batch_size=batchSize,
                                         shuffle=True)
trnEmbdngs = np.zeros((idsTrn.shape[0],hidden_size))
trnLbls  = np.zeros((idsTrn.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelSVM(inputs)
    trnEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    trnLbls[i*batchSize: (i+1)*batchSize] = labels

labels_test = torch.tensor(encoded_labels_test).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTst.int(), labels_test)), batch_size=batchSize,
                                         shuffle=True)
tstEmbdngs = np.zeros((idsTst.shape[0],hidden_size))
tstLbls  = np.zeros((idsTst.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelSVM(inputs)
    tstEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    tstLbls[i*batchSize: (i+1)*batchSize] = labels

### Linear SVM


In [ ]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo', kernel = 'linear')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyLnr = clf.score(trnEmbdngs, trnLbls)
TstAccrcyLnr = clf.score(tstEmbdngs,tstLbls)
print(r'Train Accuracy of Linear SVM =', TrnAccrcyLnr)
print(TstAccrcyLnr)

### RBF Kernel SVM

In [ ]:
clf = svm.SVC(decision_function_shape='ovo', kernel='rbf')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyKernel = clf.score(trnEmbdngs, trnLbls)
TstAccrcyKernel = clf.score(tstEmbdngs,tstLbls)
print(TrnAccrcyKernel)
print(TstAccrcyKernel)

# SVM without LSTM - Averaging Word2Vec Embeddings

In [ ]:
class w2nModelSVM(torch.nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_size, nClasses):
        super(w2nModelSVM, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        x = self.word_embeddings(x)                # output dimensions is batch size = N x sequence length x feature size
        x = torch.sum(x,dim=1)/x.shape[1]          # batch size x feature size
        return x

torch.device('cuda:0')
modelw2vAvg = w2nModelSVM(vocab_size = w2v_weights.shape[0], 
                 embedding_dim = w2v_weights.shape[1],
                 hidden_size=hidden_size, nClasses = numClasses
                )
modelw2vAvg.word_embeddings.weight.data.copy_(torch.from_numpy(w2v_weights))
modelw2vAvg.word_embeddings.weight.requires_grad=False


In [ ]:
labels_train = torch.tensor(encoded_labels_train).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTrn.int(), labels_train)),
                                         batch_size=batchSize,
                                         shuffle=True)
trnEmbdngs = np.zeros((idsTrn.shape[0],dim))
trnLbls  = np.zeros((idsTrn.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelw2vAvg(inputs)
    trnEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    trnLbls[i*batchSize: (i+1)*batchSize] = labels

labels_test = torch.tensor(encoded_labels_test).long()
trainloader = torch.utils.data.DataLoader(list(zip(idsTst.int(), labels_test)), batch_size=batchSize,
                                         shuffle=True)
tstEmbdngs = np.zeros((idsTst.shape[0],dim))
tstLbls  = np.zeros((idsTst.shape[0]))
for i, data in enumerate(trainloader):
    inputs,labels = data
    outputs = modelw2vAvg(inputs)
    tstEmbdngs[i*batchSize: (i+1)*batchSize,:] = outputs.detach().clone().numpy()
    tstLbls[i*batchSize: (i+1)*batchSize] = labels

In [ ]:
clf = svm.SVC(decision_function_shape='ovo', kernel = 'linear')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyLnr = clf.score(trnEmbdngs, trnLbls)
TstAccrcyLnr = clf.score(tstEmbdngs,tstLbls)
print(TrnAccrcyLnr)
print(TstAccrcyLnr)

In [ ]:
clf = svm.SVC(decision_function_shape='ovo', kernel='rbf')
clf.fit(trnEmbdngs, trnLbls)
TrnAccrcyKernel = clf.score(trnEmbdngs, trnLbls)
TstAccrcyKernel = clf.score(tstEmbdngs,tstLbls)
print(TrnAccrcyKernel)
print(TstAccrcyKernel)